# Winter Project

In [1]:
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

/home/aparr/anaconda3/envs/cais_tensor_2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-20 19:00:28.579694: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 19:00:28.701867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734750028.768076    1044 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734750028.780863    1044 cuda_bla

In [2]:
seed = 42

In [3]:
# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

Path to dataset files: /home/aparr/.cache/kagglehub/datasets/msambare/fer2013/versions/1


In [19]:
BATCH_SIZE = 64
from keras.utils import image_dataset_from_directory
# Import Files
train_dataset = image_dataset_from_directory(
    os.path.join(path, 'train'),
    label_mode = 'categorical',
    color_mode = 'grayscale',
    batch_size = BATCH_SIZE,
    image_size = (48, 48),
    shuffle = True,
    seed = seed
)

train_iter = iter(train_dataset)
train_labels = np.concatenate([y for x, y in train_dataset], axis=0)
train_labels = pd.from_dummies(pd.DataFrame(train_labels))[''].to_numpy() # Convert from one hot

test_dataset = image_dataset_from_directory(
    os.path.join(path, 'test'),
    label_mode = 'categorical',
    color_mode = 'grayscale',
    batch_size = BATCH_SIZE,
    image_size = (48, 48),
    shuffle = False,
)
test_iter = iter(test_dataset)
test_labels = np.concatenate([y for x, y in test_dataset], axis=0)
test_labels = pd.from_dummies(pd.DataFrame(test_labels))[''].to_numpy() # Convert from one hot

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


2024-12-20 19:11:19.120313: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [23]:
FILTERS = 4
FILTER_SZ = 5
STRIDE = (1,1)
POOL_SZ = 5
from keras import models, layers, optimizers

model = models.Sequential([
    layers.Conv2D(FILTERS, FILTER_SZ, input_shape=(48,48,1), strides=STRIDE),
    layers.MaxPooling2D(pool_size=POOL_SZ, strides=STRIDE),
    layers.Conv2D(FILTERS, FILTER_SZ, strides=STRIDE),
    layers.Conv2D(FILTERS, FILTER_SZ, strides=STRIDE),
    layers.Conv2D(FILTERS, FILTER_SZ, strides=STRIDE),
    layers.Flatten(),
    layers.Dense(1000, activation='relu'),
    layers.Dense(7, activation='softmax')
])

model.summary()

optimizer = optimizers.Adam(learning_rate=0.005)

model.compile(
    optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', 'categorical_crossentropy'],
)

/home/aparr/anaconda3/envs/cais_tensor_2/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 44, 44, 4)      │           104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 40, 40, 4)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 36, 36, 4)      │           404 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 32, 32, 4)      │           404 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 28, 28, 4)      │           404 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1000)           │     3,137,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 7)              │         7,007 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,145,323 (12.00 MB)

 Trainable params: 3,145,323 (12.00 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
EPOCHS = 10
hostory = model.fit(
    train_dataset,
    epochs = EPOCHS,
    validation_data = (test_dataset),
    
)

Epoch 1/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5702 - categorical_crossentropy: 1.1653 - loss: 1.1653 - val_accuracy: 0.3670 - val_categorical_crossentropy: 1.7757 - val_loss: 1.7757
Epoch 2/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6097 - categorical_crossentropy: 1.0606 - loss: 1.0606 - val_accuracy: 0.3638 - val_categorical_crossentropy: 1.9861 - val_loss: 1.9861
Epoch 3/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6520 - categorical_crossentropy: 0.9597 - loss: 0.9597 - val_accuracy: 0.3660 - val_categorical_crossentropy: 2.2587 - val_loss: 2.2587
Epoch 4/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6893 - categorical_crossentropy: 0.8784 - loss: 0.8784 - val_accuracy: 0.3695 - val_categorical_crossentropy: 2.2562 - val_loss: 2.2562
Epoch 5/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7173 - categorical_crossentropy: 0.8030 - loss: 0.8030 - val_accuracy: 0.3728 - val_categorical_crossentropy: 2.4706 - val_los